# キカガク「自然言語処理の基礎（PyTorch）」の学習メモ
* https://www.kikagaku.ai/tutorial/basic_of_nlp

## 導入
* 基礎技術
    * 形態素解析
        * 文章を単語ごとに切り分ける処理
    * 構文解析
        * 単語の係り受けなど、就職関係を決める処理
    * 意味解析
        * 「意味が妥当であるか」など、文章の意味を決定する処理
        * 例：望遠鏡で泳ぐ彼女を見た
    * 文脈解析
        * 複数の文について関係を決定する処理
        * 例：入り口に男性が立っている。彼の名前は佐川だ

* 自然言語処理の流れ
    * データの収集
        * 解決したいタスクに応じてデータを収集する
    * クリーニング処理→ここは実質スクレイピング
        * HTMLのタグなど意味を持たないノイズを削除する
    * 単語の正規化
        * 半角や全角、小文字大文字などを統一する
    * 形態素解析(単語分割)
        * 文章を単語ごと分割する
    * 基本形への変換
        * 語幹(活用しない部分)への統一を行う
        * 例：学ん(だ)→学ぶ
        * 昨今の実装では基本形へ変換しない場合もある
    * ストップワード除去
        * 出現回数の多すぎる単語など、役に立たない単語を除去する
        * 昨今の実装では除去しない場合もある
    * 単語の数値化
        * 機械学習で扱えるよう文字列から数値へ変換を行う
    * モデルの学習
        * タスクに合わせ、古典的な機械学習~ニューラルネットワーク選択する

* 代表的なデータセット
    * 日本政府や自治体
        * https://www.data.go.jp/
    * 教育機関
        * 東北大学 乾・鈴木研究室 Open Resources  
        * 首都大学東京 自然言語処理研究室（小町研）  
        * 京都大学 黒橋・河原・村脇研究室 自然言語処理のためのリソース
        * Graham Neubig 日本語対訳データ
    * 民間、企業
        * 青空文庫
        * livedoorニュースコーパス
        * Amazonレビューデータセット
    * Kaggle Dataset
        * https://www.kaggle.com/datasets

## ハンズオン
### 先ずはMeCab等の基本的な使い方から

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import MeCab
mecab = MeCab.Tagger('-Ochasen')

In [3]:
res = mecab.parse('こんにちは、私はキカガクです。')

In [4]:
print(res)

こんにちは	コンニチハ	こんにちは	感動詞		
、	、	、	記号-読点		
私	ワタシ	私	名詞-代名詞-一般		
は	ハ	は	助詞-係助詞		
キカガク	キカガク	キカガク	名詞-一般		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



In [5]:
text1 = 'キカガクでは、ディープラーニングを含んだ機械学習や人工知能の教育を行っています。'
text2 = '代表の吉崎は大学院では機械学習・ロボットのシステム制御、画像処理の研究に携わっていました。'
text3 = '機械学習、システム制御、画像処理ではすべて線形代数とプログラミングが不可欠になります。'

In [6]:
# 形態素解析
res = mecab.parse(text1)
print(res)

キカガク	キカガク	キカガク	名詞-一般		
で	デ	で	助詞-格助詞-一般		
は	ハ	は	助詞-係助詞		
、	、	、	記号-読点		
ディープラーニング	ディープラーニング	ディープラーニング	名詞-一般		
を	ヲ	を	助詞-格助詞-一般		
含ん	フクン	含む	動詞-自立	五段・マ行	連用タ接続
だ	ダ	だ	助動詞	特殊・タ	基本形
機械	キカイ	機械	名詞-一般		
学習	ガクシュウ	学習	名詞-サ変接続		
や	ヤ	や	助詞-並立助詞		
人工	ジンコウ	人工	名詞-一般		
知能	チノウ	知能	名詞-一般		
の	ノ	の	助詞-連体化		
教育	キョウイク	教育	名詞-サ変接続		
を	ヲ	を	助詞-格助詞-一般		
行っ	オコナッ	行う	動詞-自立	五段・ワ行促音便	連用タ接続
て	テ	て	助詞-接続助詞		
い	イ	いる	動詞-非自立	一段	連用形
ます	マス	ます	助動詞	特殊・マス	基本形
。	。	。	記号-句点		
EOS



In [7]:
nouns = [] # 品詞が名詞 (noun) である単語を格納するリスト
res = mecab.parse(text1)
words = res.split('\n')[:-2]
for word in words:
    part = word.split('\t')
    if '名詞' in part[3]:
        nouns.append(part[0])

In [8]:
nouns

['キカガク', 'ディープラーニング', '機械', '学習', '人工', '知能', '教育']

### 名刺抽出処理を関数化

In [9]:
def get_nouns(text):
    nouns = []
    res = mecab.parse(text)
    words = res.split('\n')[:-2]
    for word in words:
        part = word.split('\t')
        if '名詞' in part[3]:
            nouns.append(part[0])
    return nouns

### 基礎的なテキストデータのエンコーディング方法（ベクトル化）

* Bag of Words (Count encoding)
* tf-idf
* One-hot encoding
* Word2Vec

In [10]:
texts = [text1, text2, text3]

In [11]:
nouns_list = [ get_nouns(i) for i in texts ]
corpus = []
for nouns in nouns_list:
  corpus.append(' '.join(nouns))

In [12]:
corpus

['キカガク ディープラーニング 機械 学習 人工 知能 教育',
 '代表 吉崎 大学院 機械 学習 ロボット システム 制御 画像 処理 研究',
 '機械 学習 システム 制御 画像 処理 すべて 線形 代数 プログラミング 不可欠']

### Bag of wordsのベクトルを作る
* fit_transform() メソッドを使用すると、前述の説明通り、単語毎に ID が割り振られ、ID ごとの出現回数を元にベクトル化

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(corpus)

* vocabulary_ 属性でコーパス中のエンコーディングされた単語とその ID を確認

In [14]:
vectorizer.vocabulary_

{'キカガク': 1,
 'ディープラーニング': 3,
 '機械': 16,
 '学習': 14,
 '人工': 7,
 '知能': 18,
 '教育': 15,
 '代表': 9,
 '吉崎': 12,
 '大学院': 13,
 'ロボット': 5,
 'システム': 2,
 '制御': 11,
 '画像': 17,
 '処理': 10,
 '研究': 19,
 'すべて': 0,
 '線形': 20,
 '代数': 8,
 'プログラミング': 4,
 '不可欠': 6}

### エンコーディング後の数値を toarray() メソッドを使用して取得

* それぞれが、text1、text2、text3に対応したベクトル。
* それぞれのリストの値は出現数。リストのindexは単語のIDに対応。
* text1は`キカガク ディープラーニング 機械 学習 人工 知能 教育`で、IDに直すと`1, 3, 16, 14, 7, 18, 15`なので、リストの該当インデックスが1となっているのがわかる。
  
* [キカガクのサイト](https://www.kikagaku.ai/tutorial/basic_of_nlp/learn/pytorch_text_classification)だと下記のように書いてあるが、コーパスのIDと合っていないので間違い。（人工知能と機械学習はコーパスに存在しない。→区切りが異なる。）
```
 'キカガク': 1,
 'ディープラーニング': 3,
 '人工知能': 7,
 '教育': 12,
 '機械学習': 13, 
```

In [15]:
x.toarray()

array([[0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1]])

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### データを取ってきて、'content/text'配下に配置。
* データセットは[こちら](http://drive.google.com/uc?export=view&id=1J4N95vciIIe5w18eXpC2KYvL0DzUoITd)からダウンロード

In [17]:
from glob import glob

# ディレクトリの取得
directories = glob('content/text/*')
directories

['content/text/movie-enter',
 'content/text/it-life-hack',
 'content/text/kaden-channel',
 'content/text/topic-news',
 'content/text/livedoor-homme',
 'content/text/peachy',
 'content/text/sports-watch',
 'content/text/smax']

In [18]:
filepaths = glob('{}/*.txt'.format(directories[0]))
filepaths[:3]

['content/text/movie-enter/movie-enter-5978741.txt',
 'content/text/movie-enter/movie-enter-6322901.txt',
 'content/text/movie-enter/movie-enter-6176324.txt']

### 試しに１つファイルを読み込んで中身を見てみる

In [19]:
with open(filepaths[0], encoding='utf-8') as f:
  text = ''.join(f.readlines()[2:])
print(text)

【DVDエンター！】誘拐犯に育てられた女が目にした真実は、孤独か幸福か
　2005年11月から翌2006年7月まで読売新聞にて連載された、直木賞作家・角田光代による初の長編サスペンス『八日目の蝉』。2010年に檀れいと北乃きいの出演によりテレビドラマ化された同作が、2011年4月に永作博美と井上真央の出演によって映画化。そして、劇場公開から半年が過ぎた10月28日、DVD＆ブルーレイとなって発売されました。

八日目の蝉
　妻子ある男と愛し合い、その子を身ごもりながら、あきらめざるをえなかった女。彼女は同時に、男の妻が子供を産んだことを知る。その赤ん坊を見に行った女は、突発的にその子を連れ去り、逃避行を続けた挙句、小豆島に落ち着き、母と娘として暮らしはじめる。


不倫相手の子供を誘拐し、4年間育てた女
　永作博美が演じる野々宮希和子は、不倫相手の子を宿しながらも、彼の「いずれ妻と別れるから、それまで待ってくれ」という常套句を信じて、中絶。後遺症により、二度と子供を産めない身体となってしまいます。その後、不倫相手から彼の妻が出産したことを知らされ、別れを決意。最後に諦めをつけるため、彼らの生後6ヶ月の赤ん坊・恵理菜の顔を見た希和子でしたが、自分に笑顔で向けた恵理菜を見て、思わず誘拐。名前を変えて恵理菜を薫と名付けると、人目を避けて各地を転々とし、二人で幸せな時間を過ごしますが、辿り着いた最後の場所・小豆島で4年の逃避行に終止符を打ちます。


誘拐犯に育てられた女
　4歳になって実の両親と再会を果たした後も、世間から言われの無い中傷を受け、本当の両親との関係を築けないまま、21歳の大学生へと成長した恵理菜。過去と向き合うことを避けてきた恵理菜でしたが、劇団ひとりが演じる不倫相手・岸田孝史の子を宿し、ずっと憎み続けてきた希和子と同じ道を歩んでいることに気付いた彼女は、小池栄子が演じるルポライター・安藤千草と共に、4年間の逃亡生活を追憶する旅に出ます。希和子との幸せだった時間に触れながら、最終地・小豆島に辿り着いた恵理菜が見た真実とは？


八日目の蝉は幸せなのだろうか？
　蝉は俗説として、一生の大半を幼虫として地下で費やし、地上に出て羽化からわずか1週間程度で死ぬと言われています。八日目まで生き残ってしまった蝉が目にしたのは、孤独か、あるいは誰も目にすることのできなか

### 全記事のテキストを読み込んで、１記事単位のリストにする。同じ順番でラベル（どのソースか）のIDも振る。

In [20]:
texts, labels = [], []
for (i, directory) in enumerate(directories):
    #各ディレクトリ内のtxtファイルのパスをすべて取得
    filepaths = glob('{}/*.txt'.format(directory))
    # テキストを読み込んで、内容をtextに格納、ラベルも併せて格納
    for filepath in filepaths:
        with open(filepath, encoding='utf-8') as f:
            text = ''.join(f.readlines()[2:])  # URL等の先頭２行を除いた各行の文章を連結（join）して格納
            texts.append(text)
            labels.append(i)

In [21]:
len(texts), len(labels)

(6505, 6505)

### 各テキストから名刺を抽出して、空白区切りにしてword_colect(List)に格納

In [22]:
word_collect = []
for text in texts:
    nouns = get_nouns(text)
    word_collect.append(' '.join(nouns))

### BoW に変換

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=20)
x = vectorizer.fit_transform(word_collect)
x = x.toarray()

### PyTorchで取り扱えるデータ形式に変換
* BoWベクトル：numpy.ndarrayからtorch.tensorへ
* labels：listからtorch.tensorへ

In [24]:
import torch
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(labels, dtype=torch.int64)

### pickle化する関数を定義

In [25]:
import torch.utils.data
import pickle

def to_pickle(obj, file):
    with open(file, 'wb') as f:
        pickle.dump(obj, f, protocol=4)
        
def from_pickle(file):
    with open(file, 'rb') as f:
        obj = pickle.load(f)
    return obj

### Bowのデータとラベルデータをまとめたデータセットを作成

In [26]:
dataset = torch.utils.data.TensorDataset(x, t)
dataset

### 学習用データセットとテスト用データセットの分割

In [27]:
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

      
# データの分割
torch.manual_seed(0)

train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

to_pickle(train, 'data/train.pickle')
to_pickle(val,   'data/val.pickle')
to_pickle(test,  'data/test.pickle')

### モデルの構築

In [28]:
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [29]:
class TrainNet(pl.LightningModule):

    @pl.data_loader
    def train_dataloader(self):
        #train = from_pickle('data/train.pickle')
        return torch.utils.data.DataLoader(train, self.batch_size, shuffle=True, num_workers=self.num_workers)

    def training_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'loss': loss}
        return results

In [30]:
class ValidationNet(pl.LightningModule):

    @pl.data_loader
    def val_dataloader(self):
        #val = from_pickle('data/val.pickle')
        return torch.utils.data.DataLoader(val, self.batch_size, num_workers=self.num_workers)

    def validation_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'val_loss': loss, 'val_acc': acc}
        return results

    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        results = {'val_loss': avg_loss, 'val_acc': avg_acc}
        return results

In [31]:
class TestNet(pl.LightningModule):

    @pl.data_loader
    def test_dataloader(self):
        #test = from_pickle('data/test.pickle')
        return torch.utils.data.DataLoader(test, self.batch_size, num_workers=self.num_workers)

    def test_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'test_loss': loss, 'test_acc': acc}
        return results

    def test_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        results = {'test_loss': avg_loss, 'test_acc': avg_acc}
        return results

In [32]:
class Net(TrainNet, ValidationNet, TestNet):

    def __init__(self, batch_size=128, num_workers=8):
        super(Net, self).__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.fc1 = nn.Linear(5795, 200)
        self.fc2 = nn.Linear(200, 9)

    def lossfun(self, y, t):
        return F.cross_entropy(y, t)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [33]:
# cuDNN に対する再現性の確保
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [34]:
# 乱数のシードを固定
torch.manual_seed(0)

net = Net(batch_size=128)
trainer = Trainer(max_nb_epochs=20)

# モデルの学習
trainer.fit(net)

1

In [35]:
trainer.callback_metrics

{'loss': 3.9073142943379935e-06,
 'val_loss': 0.4451643228530884,
 'val_acc': 0.9452448487281799,
 'epoch': 19}

In [36]:
# テストデータに対する検証
trainer.test()

# テストデータに対する結果
trainer.callback_metrics

----------------------------------------------------------------------------------------------------
TEST RESULTS
{}
----------------------------------------------------------------------------------------------------



{'loss': 3.9073142943379935e-06,
 'val_loss': 0.4451643228530884,
 'val_acc': 0.9452448487281799,
 'epoch': 19,
 'test_loss': 0.30323073267936707,
 'test_acc': 0.9381425976753235}